In [ ]:
    import torch
    import torchvision
    import sys
    import torch
    import numpy as np
    import cv2
    import matplotlib.pyplot as plt
    import warnings
    import zipfile
    import io
    import glob
    import os
    !{sys.executable} -m pip install opencv-python matplotlib onnx onnxruntime
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    !mkdir images
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
    from segment_anything import sam_model_registry, SamPredictor
    from segment_anything.utils.onnx import SamOnnxModel
    import onnxruntime
    from onnxruntime.quantization import QuantType
    from onnxruntime.quantization.quantize import quantize_dynamic

In [ ]:
from google.colab import files
uploaded = files.upload()
image1_path = list(uploaded.keys())[0]
image1 = cv2.imread(image1_path)

In [ ]:
from google.colab import files
uploaded = files.upload()
lvda_mask_path = list(uploaded.keys())[0]
lvda_mask_image = cv2.imread(lvda_mask_path)

In [ ]:
checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=checkpoint)
onnx_model_path = "sam_onnx_example.onnx"
onnx_model = SamOnnxModel(sam, return_single_mask=True)
dynamic_axes = {
    "point_coords": {1: "num_points"},
    "point_labels": {1: "num_points"},
}
embed_dim = sam.prompt_encoder.embed_dim
embed_size = sam.prompt_encoder.image_embedding_size
mask_input_size = [4 * x for x in embed_size]
dummy_inputs = {
    "image_embeddings": torch.randn(1, embed_dim, *embed_size, dtype=torch.float),
    "point_coords": torch.randint(low=0, high=1024, size=(1, 5, 2), dtype=torch.float),
    "point_labels": torch.randint(low=0, high=4, size=(1, 5), dtype=torch.float),
    "mask_input": torch.randn(1, 1, *mask_input_size, dtype=torch.float),
    "has_mask_input": torch.tensor([1], dtype=torch.float),
    "orig_im_size": torch.tensor([1500, 2250], dtype=torch.float),
}
output_names = ["masks", "iou_predictions", "low_res_masks"]
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    with open(onnx_model_path, "wb") as f:
        torch.onnx.export(
            onnx_model,
            tuple(dummy_inputs.values()),
            f,
            export_params=True,
            verbose=False,
            opset_version=17,
            do_constant_folding=True,
            input_names=list(dummy_inputs.keys()),
            output_names=output_names,
            dynamic_axes=dynamic_axes,
        )

ort_session = onnxruntime.InferenceSession(onnx_model_path)

sam.to(device='cpu')
predictor = SamPredictor(sam)

In [ ]:
#создает синий контур у маски из loveDA
def get_contour_loveda_mask(path):
    loveda_mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    loveda_mask = np.where(loveda_mask == 64, 1, 0)
    loveda_mask = loveda_mask.astype(np.uint8)
    loveda_mask = loveda_mask * 255
    contours, hierarchy = cv2.findContours(loveda_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    mask_with_fill_contours = cv2.cvtColor(loveda_mask, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(mask_with_fill_contours, contours, -1, (255, 0, 0), thickness=2)

    return mask_with_fill_contours


#накладывает синий контур маски из набора на изображение
def overlay_lvda_contour(origin_img, mask):
    height, width, channels = origin_img.shape
    for y in range(height):
        for x in range(width):
            if (mask[y, x][0] > 200 and mask[y, x][1] < 10 and mask[y, x][2] < 10):
                origin_img[y, x] = np.array([0, 0, 255])

    return origin_img

def get_image4(image_path, mask_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image4 = overlay_lvda_contour(image, get_contour_loveda_mask(mask_path))
    return image4

def show_img(img):
    plt.axis('off')
    plt.imshow(img)
    plt.show()

In [ ]:
#IOU

def get_binary_lvda_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = np.where(mask == 64, 1, 0)
    mask = mask.astype(np.uint8)
    mask = mask * 255
    return mask

def get_binary_sam_mask(sam_mask):
    mask = cv2.cvtColor(sam_mask, cv2.COLOR_BGR2GRAY)
    _, bin_sam = cv2.threshold(mask, 50, 255, cv2.THRESH_BINARY)
    return bin_sam


def calculate_iou(mask_path, sam_mask):
    gt_mask = get_binary_lvda_mask(mask_path)
    pred_mask = get_binary_sam_mask(sam_mask)
    intersection = np.logical_and(gt_mask, pred_mask)
    eps = 1e-7
    union = np.logical_or(gt_mask, pred_mask) + eps
    iou_score = (np.sum(intersection) + eps) / np.sum(union)
    return round(iou_score, 2)

In [ ]:
# функция для нахождения точек принадлежащих объектам воды
def get_points_of_object(mask_path):
    points = []
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = np.where(mask == 64, 1, 0)
    mask = mask.astype(np.uint8)
    mask = mask * 255

    ret, binary_mask = cv2.threshold(mask, 30, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        try:
            black_image = np.zeros_like(mask)
            cv2.drawContours(black_image, [contour], -1, (255, 0, 0), thickness=cv2.FILLED)

            x, y, dx, dy = cv2.boundingRect(contour)
            midY = int(dy / 2 + y)
            x_values = []

            startX = x
            endX = x + dx

            for x in range(startX, endX):
                if black_image[midY, x] == 255:
                    x_values.append(x)
            sorted_x_val = sorted(x_values)
            x_arr = []
            for i in range(2, len(sorted_x_val)):
                if (sorted_x_val[i] - sorted_x_val[i - 1]) < 3:
                    x_arr.append(sorted_x_val[i])
                else:
                    break

            midX = int((x_arr[0] + x_arr[-1]) / 2)

            point = [midX, midY]
            points.append(point)

        except Exception:
            continue

    return points

In [ ]:
def check_iou_sam_contour(mask_path, point, sam_contour_mask):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask = np.where(mask == 64, 1, 0)
    mask = mask.astype(np.uint8)
    mask = mask * 255

    ret, binary_mask = cv2.threshold(mask, 30, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        distance = cv2.pointPolygonTest(contour, point, True)
        if distance > 0:
            gt_mask = np.zeros((1024, 1024), dtype=np.uint8)
            cv2.drawContours(gt_mask, [contour], contourIdx=-1, color=255, thickness=cv2.FILLED)

            gt_mask = gt_mask.astype(np.uint8)
            gt_mask = gt_mask * 255

            pred_mask = sam_contour_mask

            intersection = np.logical_and(gt_mask, pred_mask)
            eps = 1e-7
            union = np.logical_or(gt_mask, pred_mask) + eps
            iou_score = (np.sum(intersection) + eps) / np.sum(union)
            idx_iou = round(iou_score, 2)

            if idx_iou < 0.4:
                return gt_mask

            break

    return pred_mask

In [ ]:
def get_sam_mask(image, mask_path):
    predictor.set_image(image)
    image_embedding = predictor.get_image_embedding().cpu().numpy()
    image_embedding.shape
    masks_arr = []
    points = get_points_of_object(mask_path)


    for point in points:
        input_point = np.array([point])
        input_label = np.array([1])

        onnx_coord = np.concatenate([input_point, np.array([[0.0, 0.0]])], axis=0)[None, :, :]
        onnx_label = np.concatenate([input_label, np.array([-1])], axis=0)[None, :].astype(np.float32)
        onnx_coord = predictor.transform.apply_coords(onnx_coord, image.shape[:2]).astype(np.float32)

        onnx_mask_input = np.zeros((1, 1, 256, 256), dtype=np.float32)
        onnx_has_mask_input = np.zeros(1, dtype=np.float32)

        ort_inputs = {
            "image_embeddings": image_embedding,
            "point_coords": onnx_coord,
            "point_labels": onnx_label,
            "mask_input": onnx_mask_input,
            "has_mask_input": onnx_has_mask_input,
            "orig_im_size": np.array(image.shape[:2], dtype=np.float32)
        }

        masks, _, low_res_logits = ort_session.run(None, ort_inputs)
        masks = masks > predictor.model.mask_threshold

        masks.shape
        mask = np.squeeze(masks)
        check_mask = check_iou_sam_contour(mask_path, point, mask)# здесь сделать проверку iou
        masks_arr.append(check_mask)

    color = np.array([30, 144, 255], dtype=np.float32)
    sam_mask = np.zeros_like(image, dtype=np.float32)

  # здесь все маски накладываются
    for mask in masks_arr:
        show_img(mask)
        sam_mask[mask == 1] = color

    sam_mask = sam_mask.astype('uint8')

    return sam_mask


In [ ]:
#создает красный контур у маски sam
def get_contour_sam_mask(mask_image):
    gray = cv2.cvtColor(mask_image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(mask_image, contours, -1, (255, 0, 0), 2)
    return mask_image

#накладывает красный контур маски sam на изображение
def overlay_mask_contour(origin_img, mask):
    height, width, channels = origin_img.shape
    for y in range(height):
        for x in range(width):
            if mask[y, x][0] >= 200 and mask[y, x][1] <= 50 and mask[y, x][2] <= 50:
                origin_img[y, x] = np.array([255, 0, 0])

    return origin_img

def get_image3(image_path, mask_path, sam_mask):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image3 = overlay_mask_contour(image, get_contour_sam_mask(sam_mask))
    return image3

In [ ]:
#создает красную заполненную маску из набора
def get_contour_mask(mask_path):
    image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    image = np.where(image == 64, 1, 0)
    image = image.astype(np.uint8)
    image = image * 255

    ret, binary_mask = cv2.threshold(image, 40, 100, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask_with_fill_contours = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(mask_with_fill_contours, contours, -1, (0, 0, 255), thickness=cv2.FILLED)

    return mask_with_fill_contours

#накладывает прозрачную белую маску из набора
def overlay_transparent_loveda_mask(image_path, mask_path):
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    mask = get_contour_mask(mask_path)

    red_mask = cv2.inRange(mask, (0, 0, 200), (100, 100, 255))
    red_mask = cv2.cvtColor(red_mask, cv2.COLOR_GRAY2BGR)

    result = cv2.addWeighted(original_image, 1, red_mask, 0.4, 0)

    return result

#накладывает красный контур sam
def overlay_sam_contour(origin_img, mask):
    height, width, channels = origin_img.shape
    for y in range(height):
        for x in range(width):
            if (mask[y, x][0] > 200 and mask[y, x][1] < 10 and mask[y, x][2] < 10):
                origin_img[y, x] = np.array([255, 0, 0])

    return origin_img


def get_image2(image_path, mask_path, img3):
    img2 = overlay_transparent_loveda_mask(image_path, mask_path)
    image2 = overlay_sam_contour(img2, img3)
    return image2

In [ ]:
def get_sam_mask_for_folder(sam_mask):
    mask = get_binary_sam_mask(sam_mask)
    mask[mask == 255] = 64
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    return mask

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
uploaded1 = files.upload()

In [ ]:
zip_file = zipfile.ZipFile(io.BytesIO(uploaded['images.zip']), 'r')
zip_file.extractall('images_folder')

zip_file1 = zipfile.ZipFile(io.BytesIO(uploaded1['masks.zip']), 'r')
zip_file1.extractall('masks_folder')

In [ ]:
res_fold = 'images'
os.makedirs(res_fold, exist_ok=True)

masks_folder = 'gt2'
os.makedirs(masks_folder, exist_ok=True)

In [ ]:
images = glob.glob('images_folder/images/*.tif')
masks = glob.glob('masks_folder/masks/*.tif')

i = 0
for image in images:
    img_name = image.split('/')[-1]
    for mask in masks:
        msk_name = mask.split('/')[-1]
        if img_name == msk_name:
            i+=1
            print(i)
            img1 = cv2.imread(image)
            points = get_points_of_object(mask)

            #если есть объекты воды, то обрабатываем картинки и сохраняем
            if points:
                sam_mask = get_sam_mask(img1, mask)

                img4 = get_image4(image, mask)
                img3 = get_image3(image, mask, sam_mask)
                img2 = get_image2(image, mask, img3)

                img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
                img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
                img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)

                iou = calculate_iou(mask, sam_mask)

                #сохранение обработанных картинок в папку
                img1_path = os.path.join(res_fold, img_name.split('.')[0]+'_1_iou_' + str(iou) + '.tif')
                cv2.imwrite(img1_path, img1)

                img2_path = os.path.join(res_fold, img_name.split('.')[0]+'_2_iou_' + str(iou) + '.tif')
                cv2.imwrite(img2_path, img2)

                img3_path = os.path.join(res_fold, img_name.split('.')[0]+'_3_iou_' + str(iou) + '.tif')
                cv2.imwrite(img3_path, img3)

                img4_path = os.path.join(res_fold, img_name.split('.')[0]+'_4_iou_' + str(iou) + '.tif')
                cv2.imwrite(img4_path, img4)

                #сохранение маски сэм в папку
                mask = get_sam_mask_for_folder(sam_mask)
                mask_path = os.path.join(masks_folder, img_name)
                cv2.imwrite(mask_path, mask)
            else:
                img1_path = os.path.join(res_fold, img_name.split('.')[0] + '.tif')
                cv2.imwrite(img1_path, img1)

In [ ]:
zipf = zipfile.ZipFile(res_fold + '.zip', 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk(res_fold):
    for file in files:
        zipf.write(os.path.join(root, file))
zipf.close()

In [ ]:
from google.colab import files

files.download(res_fold + '.zip')

In [ ]:
zipf_m = zipfile.ZipFile(masks_folder + '.zip', 'w', zipfile.ZIP_DEFLATED)
for root, dirs, files in os.walk(masks_folder):
    for file in files:
        zipf_m.write(os.path.join(root, file))
zipf_m.close()


In [ ]:
from google.colab import files

files.download(masks_folder + '.zip')